# 特征有效性分析

- 先验概率：不考虑特征，只考虑结果
- 后验概率：（特征相对于某个标签的后验概率）即条件概率

## 不确定性的减少可以作为特征有效性分析的一个指标

## 确定性的增加可以作为特征有效性分析的一个指标

# 贝叶斯与逻辑回归比较

权重不同，原因：

    引入了两个独立条件假设，贝叶斯方法直接跳过了逻辑回归中反复迭代用梯度下降法才能求出的权重
    附加假设：可以直接通过统计结果求权重，而不必用梯度下降法
    在某种假定下，可以证明：与朴素贝叶斯分类方法一样，许多神经网络和曲线拟合算分输出最大的后验假定

样本数量不够，特征数量过多，容易引起过拟合

# 处理类别特征

## 类别型变量特征

In [3]:
# create a dataframe with an integer feature and a categorical string feature
import pandas as pd

demo_df = pd.DataFrame({"Integer Feature": [0, 1, 2, 1], "Categorical Feature": ["socks", "fox", "socks", "box"]})
demo_df

,Categorical Feature,Integer Feature
0,socks,0
1,fox,1
2,socks,2
3,box,1


get_dummies对“整数特征”无变化，对类别特征进行one-hot编码

In [4]:
pd.get_dummies(demo_df)

,Integer Feature,Categorical Feature_box,Categorical Feature_fox,Categorical Feature_socks
0,0,0.0,0.0,1.0
1,1,0.0,1.0,0.0
2,2,0.0,0.0,1.0
3,1,1.0,0.0,0.0


将“整形特征”变为“字符型类别”进行one-hot编码

In [5]:
demo_df["Integer Feature"] = demo_df["Integer Feature"].astype(str)
pd.get_dummies(demo_df)

,Categorical Feature_box,Categorical Feature_fox,Categorical Feature_socks,Integer Feature_0,Integer Feature_1,Integer Feature_2
0,0.0,0.0,1.0,1.0,0.0,0.0
1,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,1.0,0.0


## 连续型变量特征